# Constructing RAG application with knowledge graph

In [1]:
from langchain.graphs import Neo4jGraph
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
import os

/Users/albertebaht/Desktop/DTU/9_semester/02456 Deep learning/02456-deeplearning-project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
os.environ["NEO4J_URI"] = os.getenv('NEO4J_URI')
os.environ["NEO4J_USERNAME"] = os.getenv('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"] = os.getenv('NEO4J_PASSWORD')
os.environ["OPENAI_API_KEY"] =  os.getenv('OPENAI_API_KEY')
NEO4j_URI = os.getenv('NEO4J_URI')
NEO4j_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4j_PASSWORD = os.getenv('NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

graph = Neo4jGraph(url = NEO4j_URI, username=NEO4j_USERNAME, password = NEO4j_PASSWORD, refresh_schema=False)

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    url=NEO4j_URI,
    username=NEO4j_USERNAME,
    password= NEO4j_PASSWORD,
    node_label="KeyElement", #["Document","Chunk", "AtomicFact", "KeyElement"],
    text_node_properties=["id"],#["id", "text", "index"], #['name', 'description', 'status'],
    embedding_node_property="embedding", #'embedding'
)

/var/folders/6x/y222w8z11pdfzwfdxt9611kr0000gn/T/ipykernel_38194/3832749070.py:10: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url = NEO4j_URI, username=NEO4j_USERNAME, password = NEO4j_PASSWORD, refresh_schema=False)
/var/folders/6x/y222w8z11pdfzwfdxt9611kr0000gn/T/ipykernel_38194/3832749070.py:13: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  OpenAIEmbeddings(op

In [3]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(), chain_type="stuff", retriever=vector_index.as_retriever())

In [4]:
vector_qa.invoke(
    {"query": "How will recommendation service be updated?"}
)

{'query': 'How will recommendation service be updated?',
 'result': 'The recommendation service can be updated through collaborative prediction, where user preferences and behaviors are analyzed to make predictions on what they might like. This can help improve the accuracy of recommendations and enhance user satisfaction. Additionally, incorporating user feedback and constantly improving the algorithm can also help in updating the recommendation service.'}

In [9]:
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini"),
    qa_llm = ChatOpenAI(temperature=0), graph=graph, verbose=True, allow_dangerous_requests=True
)

In [10]:
cypher_chain.invoke(
    {"query": "How many open tickets there are?"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Document)-[:HAS_CHUNK]->(c:Chunk)-[:HAS_ATOMIC_FACT]->(af:AtomicFact)
RETURN COUNT(af) AS open_tickets

Full Context:
[{'open_tickets': 6379}]

> Finished chain.


{'query': 'How many open tickets there are?',
 'result': 'There are 6379 open tickets.'}

In [ ]:
cypher_chain.invoke(
    {"query": "Which document has most connections to other documents?"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Document)-[:HAS_CHUNK]->(c:Chunk)-[:NEXT]->(c2:Chunk)<-[:HAS_CHUNK]-(d2:Document)
RETURN d.id, COUNT(d2) AS connections
ORDER BY connections DESC
LIMIT 1

Full Context:
[{'d.id': '1611.04701v2.pdf.json', 'connections': 308}]

> Finished chain.


{'query': 'Which document has most connections to other docuuments?',
 'result': 'The document with the ID 1611.04701v2.pdf.json has the most connections to other documents.'}

In [15]:
from langchain.agents import create_openai_functions_agent, Tool, AgentExecutor
from langchain import hub


tools = [
    Tool(
        name="Tasks",
        func=vector_qa.invoke,
        description="""Useful when you need to answer questions about descriptions of tasks.
        Not useful for counting the number of tasks.
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.invoke,
        description="""Useful when you need to answer questions about microservices,
        their dependencies or assigned people. Also useful for any sort of
        aggregation like counting the number of tasks, etc.
        Use full question as input.
        """,
    ),
]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(
    ChatOpenAI(temperature=0, model_name='gpt-4o-mini'), tools, prompt
)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/Users/albertebaht/Desktop/DTU/9_semester/02456 Deep learning/02456-deeplearning-project/.venv/lib/python3.9/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [16]:
response = agent_executor.invoke({"input": "Which team is assigned to maintain PaymentService?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `Which team is assigned to maintain PaymentService?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Document)-[:HAS_CHUNK]->(c:Chunk)-[:HAS_ATOMIC_FACT]->(af:AtomicFact)
WHERE af.text CONTAINS 'PaymentService'
RETURN d

Full Context:
[]

> Finished chain.
{'query': 'Which team is assigned to maintain PaymentService?', 'result': "I don't know the answer."}I don't have information on which team is assigned to maintain PaymentService.

> Finished chain.
{'input': 'Which team is assigned to maintain PaymentService?', 'output': "I don't have information on which team is assigned to maintain PaymentService."}
